# Automated extract SVC data and compared with MX camera

In [1]:
#Generic imports
import pandas as pd
import geopandas as gpd
import numpy as np
import os
from os.path import exists
import matplotlib.pyplot as plt
import seaborn as sb
from rasterstats import zonal_stats

#Local imports
from specdal import Collection, Spectrum

## Asociar los ficheros a las ids de los plots

In [2]:
def relate_index_with_file_name_GLOBAL(datadir):
    file_exists = exists(datadir+'files_and_ids_plot.csv')
    if file_exists:
        os.remove(datadir+'files_and_ids_plot.csv')
    #Extraer lista de rutas
    list_rutas = []
    df_global  = pd.DataFrame(columns = ['filas','columnas','id','file_names','tipo'])
    for folder_tipo in os.listdir(datadir)[0:2]:
        for folder in os.listdir(datadir + folder_tipo):
            #list_rutas.append(datadir+folder_tipo+'/'+folder+'/')
            df = relate_index_with_file_name(datadir, folder_tipo, folder)
            df_global = pd.concat([df_global, df])
            #df_global = df_global.concat(df)
    #Escribir en un CSV
    df_global.to_csv(datadir+'files_and_ids_plot.csv', mode='x')
    return df_global
            
    
def relate_index_with_file_name(datadir, tipo, folder):
        
    datadir = datadir + '/' + tipo + '/' + folder
    #Proceso
    fila_inicial = int(str(folder[14:16]))
    fila_final = int(str(folder[17:19]))
    columna = int(str(folder[7:9]))

    if fila_inicial > fila_final:
        step = -1
    else:
        step = 1

    filas = np.arange(fila_inicial, fila_final-1, step, dtype='int')
    columnas = np.ones(len(filas), dtype='int')*columna
    #Convert to dataframe
    df_filas = pd.DataFrame(filas, columns=['filas'])
    df_columnas = pd.DataFrame(columnas, columns=['columnas'])
    #Merge dataframes
    df_data = df_filas.merge(df_columnas, how='left', left_index=True, right_index=True)
    #Calculate id with columns*1000 + rows
    df_data['id'] = df_data['columnas']*1000 + df_data['filas']

    #Asociar nombre fichero a la id
    files_names = []
    for f in sorted(os.listdir(datadir)):
        if f.endswith('.sig'):
            files_names.append(f[0:-4])
    #Eliminar los target del panel en blanco y de los carteles informativos en el inicio y el fin
    del files_names[0:2]    #Target del panel y cartel informativo inicio     
    del files_names[-1]     #Cartel informativo final

    #Añadir el nombre del fichero al dataframe
    df_data['file_name'] = files_names
    #Añadir el tipo........ESTO FFALTA POR HACER
    df_data['tipo'] = tipo
    return df_data

if __name__ == "__main__":
    datadir = "Z:/11-Projects/CERESTRES/04-Raw/SVC_HR1024i/Santaella/2022_04_22/classified/"
    df_measures = relate_index_with_file_name_GLOBAL(datadir)

## Unir las medidas con los atributos de los plots

In [3]:
#Path for shapefile templates
att_dir_riego="Z:/11-Projects/CERESTRES/02-ensayo/Santaella/Shapefiles/01-attributes/riego/riego_shapefile_full_atributes.shp"
att_dir_secano="Z:/11-Projects/CERESTRES/02-ensayo/Santaella/Shapefiles/01-attributes/secano/secano_shapefile_full_atributes.shp"
#Read shapefile with geopandas
df_riego_att = gpd.read_file(att_dir_riego).set_index('id_real').rename(columns={"tipo":"tipo_plantilla", "fila_real":"fila_plantilla", 
                                                                                 "columna_re":"columna_plantilla", "numero_var":"numero_variedad", 
                                                                                 "nombre_var":"nombre_variedad"})
df_secano_att = gpd.read_file(att_dir_secano).set_index('id_real').rename(columns={"tipo":"tipo_plantilla", "fila_real":"fila_plantilla", 
                                                                                 "columna_re":"columna_plantilla", "numero_var":"numero_variedad", 
                                                                                 "nombre_var":"nombre_variedad"})
#Divide original df into riego and secano for merging
df_measures_riego = df_measures.loc[df_measures['tipo']=='riego'].set_index('id')
df_measures_secano = df_measures.loc[df_measures['tipo']=='secano'].set_index('id')

# Merge dataframes
df_riego = pd.merge(df_measures_riego, df_riego_att, how='inner', right_index=True, left_index=True)
df_secano = pd.merge(df_measures_secano, df_secano_att, how='inner', right_index=True, left_index=True)

df_measures_att = pd.concat([df_riego, df_secano])

C:\Users\sergi\anaconda3\envs\specdal\lib\site-packages\pandas\core\indexes\base.py:6982: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)


## Extraer metadata, data con Specdal y unir los metadatos con las medidas realizadas

In [4]:
def extract_metadata_and_data_SVC(data_path):
    datadir =data_path
    c = Collection(name='santaella')
    metadata = []
    for f in sorted(os.listdir(datadir)):
        if f.endswith('.sig'):
            spectrum = Spectrum(filepath=os.path.join(datadir, f))
            c.append(spectrum)
            df = pd.DataFrame([spectrum.metadata])
            metadata.append(df)
    df_metadata = pd.concat(metadata)

    return df_metadata, c.data


if __name__ == '__main__':
    data_path = "Z:/11-Projects/CERESTRES/04-Raw/SVC_HR1024i/Santaella/2022_04_22/"
    export_path = data_path
    #Call function
    metadata, data = extract_metadata_and_data_SVC(data_path=data_path)
    #Exportar a un CSV (df_metadata y data)
    metadata.to_csv(export_path+'metadata.csv', encoding='utf-8', index=False)
    data.to_csv(export_path+'data.csv', encoding='utf-8')
    
    #SE UNEN LOS DATAFRAMES DE MEDIDAS SVC Y EL DE METADATOS PARA TENER SOLAMENTE LAS MEDIDAS DE LOS PLOTS EN UN CSV (metadata_filter.csv)
    df_measures_att = df_measures_att.reset_index().rename(columns={"index":"id"}).set_index('file_name')
    metadata = metadata.set_index('file_name')

    metadata_filter = pd.merge(metadata,df_measures_att,  how='inner', left_index=True, right_index=True)
    metadata_filter = metadata_filter.drop(['file_names'], axis=1).reset_index()
    #Filtrar el dataset de data con los ficheros exclusivamente de las medidas
    data_filter = data[metadata_filter['file_name']]
    
    # Write to a CSV file
    data_filter.to_csv(export_path+'data_filter.csv', encoding='utf-8', index=False)
    metadata_filter.to_csv(export_path+'metadata_filter.csv', encoding='utf-8', index=False)

In [37]:
metadata_filter['file_name']

0     220421_1139_R011_T014
1     220421_1140_R011_T015
2     220421_1141_R011_T016
3     220421_1141_R011_T017
4     220421_1141_R011_T018
              ...          
88    220421_1223_R108_T122
89    220421_1224_R108_T123
90    220421_1224_R108_T124
91    220421_1224_R108_T125
92    220421_1225_R108_T126
Name: file_name, Length: 93, dtype: object

In [32]:
data[metadata_filter['file_name']]

,220421_1139_R011_T014,220421_1140_R011_T015,220421_1141_R011_T016,220421_1141_R011_T017,220421_1141_R011_T018,220421_1142_R011_T019,220421_1142_R011_T020,220421_1143_R011_T021,220421_1143_R011_T022,220421_1144_R011_T023,...,220421_1222_R108_T117,220421_1222_R108_T118,220421_1223_R108_T119,220421_1223_R108_T120,220421_1223_R108_T121,220421_1223_R108_T122,220421_1224_R108_T123,220421_1224_R108_T124,220421_1224_R108_T125,220421_1225_R108_T126
wavelength,,,,,,,,,,,,,,,,,,,,,
338.0,0.0065,0.0104,0.0147,0.0142,0.0117,0.0092,0.0116,0.0118,0.0145,0.0097,...,0.0088,0.0059,0.0106,0.0085,0.0074,0.0053,0.0068,0.0119,0.0089,0.0062
339.4,0.0064,0.0102,0.0146,0.0141,0.0114,0.0091,0.0114,0.0120,0.0145,0.0095,...,0.0089,0.0059,0.0105,0.0085,0.0074,0.0050,0.0067,0.0120,0.0087,0.0062
340.9,0.0064,0.0102,0.0146,0.0141,0.0116,0.0092,0.0114,0.0118,0.0146,0.0095,...,0.0090,0.0059,0.0106,0.0085,0.0074,0.0050,0.0068,0.0119,0.0087,0.0062
342.4,0.0063,0.0103,0.0146,0.0141,0.0115,0.0091,0.0113,0.0119,0.0144,0.0094,...,0.0089,0.0060,0.0106,0.0085,0.0074,0.0051,0.0068,0.0119,0.0085,0.0061
343.8,0.0064,0.0102,0.0148,0.0141,0.0115,0.0090,0.0114,0.0119,0.0143,0.0095,...,0.0088,0.0059,0.0107,0.0085,0.0074,0.0052,0.0068,0.0120,0.0086,0.0062
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2509.9,0.0137,0.0183,0.0320,0.0137,0.0091,0.0091,0.0320,0.0091,0.0183,0.0320,...,0.0046,0.0000,0.0230,0.0046,0.0092,0.0000,0.0138,0.0184,-0.0046,0.0046
2512.2,0.0204,0.0153,0.0459,0.0051,0.0102,0.0102,0.0306,0.0204,0.0357,0.0459,...,0.0206,0.0155,0.0206,-0.0155,0.0155,0.0052,0.0103,0.0103,0.0052,0.0052
2514.5,0.0066,0.0329,0.0395,0.0197,0.0197,0.0132,0.0263,0.0329,0.0329,0.0395,...,0.0274,0.0137,0.0068,0.0274,0.0000,0.0000,0.0068,0.0000,0.0068,0.0137


## Generar un report - comparar los datos con los de la cámara multiespectral

In [2]:
 datadir_classified_measures = "Z:/11-Projects/CERESTRES/04-Raw/SVC_HR1024i/Santaella/2022_04_22/classified/"

In [5]:
datadir_classified_measures[0:-11]

'Z:/11-Projects/CERESTRES/04-Raw/SVC_HR1024i/Santaella/2022_04_22/'

In [6]:
data

NameError: name 'data' is not defined